# Imports

In [ ]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), os.pardir))

In [ ]:
from Tools.leica_tools import RawLoader
from Tools.sample_tools import Sample
from Tools.db_tools import DbManager


# Data prep

In [ ]:
expID = 'NKIP_FA_067'
rawloader = RawLoader(expID)
rawloader.frame_df

# Droplet detection

Execute a preview run of the droplet detection. An image will be saved to the exp folder in the analyses directory. 
If droplets are not well detected consider changing the droplet size estimate in setup.xlsx (re-run RawLoader API).

In [ ]:
frameID = 0
sample = Sample(expID, frameID)
sample.detect_droplets(mode='sweep')
sample.visualize_droplets(channel=0,save=True)

Run droplet detection through all frames of the experiment. drop_register.csv will be created at the end of the process.

In [ ]:
dbm = DbManager()
dbm.generate_drop_register(expID, mode='sweep')
dbm.generate_tfrecord(expID)

# Outlier detection

In [ ]:
expID = 'NKIP_FA_058'
dbm = DbManager()
dbm.detect_outliers(expID, model_name='outlier_v2.h5')

In [ ]:
sample = Sample(expID, 0)
sample.reload_droplets()
sample.visualize_droplets(channel=0)

# Workpackage Generation

In [ ]:
dbm = DbManager()
dbm.generate_wp(expID='NKIP_FA_058', exclude_query='outlier == True')

# Cell counting

In [ ]:
dbm = DbManager()
dbm.cell_count(expID='NKIP_FA_058', model_name='cell_count_v2.h5')